In [1]:
!pip install -qU datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which

In [2]:
from datasets import load_dataset
import torch

In [ ]:
from datasets import load_dataset

dataset = load_dataset("code-search-net/code_search_net", split="train[:20]", trust_remote_code=True)

# Lowering test dataset size to train faster
# datasetDictionary = dataset.train_test_split(test_size=0.004)

Generating train split:   0%|          | 0/1880853 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100529 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/89154 [00:00<?, ? examples/s]

In [ ]:
dataset

In [ ]:
def promptFormattingFunction() :
    pass

In [ ]:
dataset = dataset.map(promptFormattingFunction, batched=True,)